In [3]:
import pandas as pd

from data_cleaning import fetch_and_clean_data

tetouan = fetch_and_clean_data()

c:\Users\bfran\Ironhack\Week 7\powerhouse-squad-project\data_cleaning.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'DateTime_cleaned'] = pd.to_datetime(X['DateTime'])
c:\Users\bfran\Ironhack\Week 7\powerhouse-squad-project\data_cleaning.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'DayOfWeek'] = X['DateTime_cleaned'].dt.day_name()
c:\Users\bfran\Ironhack\Week 7\powerhouse-squad-project\data_cleaning.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [4]:
tetouan['Month'].value_counts()

Month
1     4464
3     4464
5     4464
7     4464
8     4464
10    4464
4     4320
6     4320
9     4320
11    4320
12    4320
2     4032
Name: count, dtype: int64

In [5]:
tetouan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52416 entries, 0 to 52415
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Month                  52416 non-null  int32  
 1   Monday                 52416 non-null  int32  
 2   Tuesday                52416 non-null  int32  
 3   Wednesday              52416 non-null  int32  
 4   Thursday               52416 non-null  int32  
 5   Friday                 52416 non-null  int32  
 6   Saturday               52416 non-null  int32  
 7   Sunday                 52416 non-null  int32  
 8   Morning                52416 non-null  int32  
 9   Afternoon              52416 non-null  int32  
 10  Evening                52416 non-null  int32  
 11  Night                  52416 non-null  int32  
 12  Temp                   52416 non-null  float64
 13  Humidity               52416 non-null  float64
 14  Wind_Speed             52416 non-null  float64
 15  ge

In [6]:
from sklearn.model_selection import train_test_split
features = tetouan.drop(columns = ["Zone_1_PC","Zone_2_PC","Zone_3_PC"])
target = tetouan[["Zone_1_PC","Zone_2_PC","Zone_3_PC"]]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [8]:
X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_train_scaled.head()

,Month,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Morning,Afternoon,Evening,Night,Temp,Humidity,Wind_Speed,general diffuse flows,diffuse flows
0,0.909091,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.322961,0.901749,0.003575,0.000031,0.000088
1,0.363636,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.583549,0.382219,0.004974,0.773860,0.069434
2,0.272727,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.340641,0.947280,0.003575,0.000047,0.000135
3,0.363636,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.483448,0.736401,0.003109,0.653481,0.304265
4,0.818182,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.404836,0.895758,0.756257,0.000046,0.000091


In [9]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)
X_test_scaled.head()

,Month,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Morning,Afternoon,Evening,Night,Temp,Humidity,Wind_Speed,general diffuse flows,diffuse flows
0,0.454545,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.468215,0.938893,0.756412,0.000044,0.000139
1,0.818182,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.471751,0.690391,0.757034,0.000063,0.000111
2,0.727273,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.646384,0.424635,0.757345,0.674118,0.070983
3,0.545455,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.591709,0.604960,0.755324,0.514530,0.106507
4,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.179610,0.712437,0.005130,0.000040,0.000111


In [11]:
# K Nearest Neighbors Test for unscaled data


# Import K Nearest Neighbors Regressor
from sklearn.neighbors import KNeighborsRegressor

# defines our knnregressor, using 10 neighbors
knnregress = KNeighborsRegressor(n_neighbors=10)

# Fits KNN to scaled data
knnregress.fit(X_train, y_train)

# Tests Model's performance
knnregress.score(X_test, y_test)

0.702652544343792

In [10]:
# K Nearest Neighbors Test for scale data


# Import K Nearest Neighbors Regressor
from sklearn.neighbors import KNeighborsRegressor

# defines our knnregressor, using 10 neighbors
knnregress = KNeighborsRegressor(n_neighbors=10)

# Fits KNN to scaled data
knnregress.fit(X_train_scaled, y_train)

# Tests Model's performance
knnregress.score(X_test_scaled, y_test)

# The model used on scaled data is much stronger. 

0.9113606511292204

In [13]:
# linear regression
# import packages and measure
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# define linear regression model
lin_reg = LinearRegression()

# fits data to a linear regression
lin_reg.fit(X_train_scaled, y_train)

pred = lin_reg.predict(X_test_scaled)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", lin_reg.score(X_test_scaled, y_test))

# This R2 of 0.50 is way lower than our K Nearest Neighbors model strength, so this is not the way to go. 

MAE 3539.94848909618
RMSE 4421.736415705596
R2 score 0.5097780915508853


In [14]:
# Decision Tree
# Import package
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=10)

# fit data to tree model
tree.fit(X_train_scaled, y_train)

pred = tree.predict(X_test_scaled)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", tree.score(X_test_scaled, y_test))

# 0.77 is better than the linear regression, but still not nearly as good as K Nearest Neighbors

MAE 2050.0224612598645
RMSE 2965.590906985771
R2 score 0.7754143025616385


In [16]:
# Ensemble Methods
# Import Packages
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

# Import Measures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [17]:
# Bagging and Pasting

# Define Model
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

# Fit data to model
bagging_reg.fit(X_train_scaled, y_train)

# Set predictor
pred = bagging_reg.predict(X_test_scaled)

# Evaluate Results
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_scaled, y_test))

# Slightly less accurate than Decision Tree

MAE 2143.2852671552105
RMSE 2985.623182895992
R2 score 0.7725803442485933


In [18]:
# Random Patches
# Initialize Random Forest
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

# Train model
forest.fit(X_train_scaled, y_train)

# Set predictor
pred = forest.predict(X_test_scaled)

# Evaluate Model
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test_scaled, y_test))

# Best Model yet - 0.9198 

MAE 1067.2477457444675
RMSE 1770.8268647677105
R2 score 0.9198516869108654


In [21]:
# AdaBoost
# Initialize AdaBoost model
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

# AdaBoost Model needs to have just one regressor variable to look at, not 3
# Create subsets
y_train_1 = y_train.drop(columns=['Zone_2_PC','Zone_3_PC'])
y_train_2 = y_train.drop(columns=['Zone_1_PC','Zone_3_PC'])
y_train_3 = y_train.drop(columns=['Zone_1_PC','Zone_2_PC'])
y_test_1 = y_test.drop(columns=['Zone_2_PC','Zone_3_PC'])
y_test_2 = y_test.drop(columns=['Zone_1_PC','Zone_3_PC'])
y_test_3 = y_test.drop(columns=['Zone_1_PC','Zone_2_PC'])

# Train Model 1
ada_reg.fit(X_train_scaled, y_train_1)

# Set predictor
pred = ada_reg.predict(X_test_scaled)

# Evaluate model 1
print("AdaBoost Model 1 MAE:", mean_absolute_error(pred, y_test_1))
print("AdaBoost Model 1 RMSE:", mean_squared_error(pred, y_test_1, squared=False))
print("AdaBoost Model 1 R2 score:", ada_reg.score(X_test_scaled, y_test_1))

# Train Model 2
ada_reg.fit(X_train_scaled, y_train_2)

# Set predictor
pred = ada_reg.predict(X_test_scaled)

# Evaluate model 2
print("AdaBoost Model 2 MAE:", mean_absolute_error(pred, y_test_2))
print("AdaBoost Model 2 RMSE:", mean_squared_error(pred, y_test_2, squared=False))
print("AdaBoost Model 2 R2 score:", ada_reg.score(X_test_scaled, y_test_2))

# Train Model 3
ada_reg.fit(X_train_scaled, y_train_3)

# Set predictor
pred = ada_reg.predict(X_test_scaled)

# Evaluate model 3
print("AdaBoost Model 3 MAE:", mean_absolute_error(pred, y_test_3))
print("AdaBoost Model 3 RMSE:", mean_squared_error(pred, y_test_3, squared=False))
print("AdaBoost Model 3 R2 score:", ada_reg.score(X_test_scaled, y_test_3))

# This took 1m10s to run.
# Model 1 R2 is 0.9300, best yet. 
# Model 2 R2 is 0.9427, best again.
# Model 3 R2 is 0.9648, highest yet. 

c:\Users\bfran\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoost Model 1 MAE: 1097.2474727325994
AdaBoost Model 1 RMSE: 1893.1015616162888
AdaBoost Model 1 R2 score: 0.9300068292867248


c:\Users\bfran\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoost Model 2 MAE: 701.7270442602472
AdaBoost Model 2 RMSE: 1243.5982937235415
AdaBoost Model 2 R2 score: 0.9426631748497314


c:\Users\bfran\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoost Model 3 MAE: 697.935171413906
AdaBoost Model 3 RMSE: 1256.3769520830983
AdaBoost Model 3 R2 score: 0.9648247054017346


In [28]:
# Gradient Boosting
# ChatGPT says this could take from 1 to 3 hours to run. Fun!

# Import package
from sklearn.multioutput import MultiOutputRegressor

# Initialize model
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

# Set up a multi-output regressor since our target has multiple columns
mor = MultiOutputRegressor(gb_reg)

# Train Model
mor.fit(X_train_scaled, y_train)

# Set Predictor
pred = mor.predict(X_test_scaled)

# Evaluate Model
print("MAE", mean_absolute_error(pred, y_test, multioutput='raw_values'))
print("RMSE", mean_squared_error(pred, y_test, multioutput='raw_values', squared=False))
print("R2 score", mor.score(X_test_scaled, y_test))

# I have no idea why it took Rosemary 30 minutes to run this, cus it ran for me in 2 minutes. 
# R2 of 0.9029 puts this in like 3rd place overall. 

MAE [1388.57033758  920.76778532  900.68079515]
RMSE [2512.94412151 1676.6143022  1692.59847341]
R2 score 0.902869825700234
